In [ ]:
sehir = 'İstanbul'
unvan = 'Game Developer'

from googlesearch import search
url=search('site:github.com AND '+unvan+' AND '+sehir,num_results=105)
clean_urls=[]

for raw_url in url:
    try:
        raw_url=raw_url.split('/')
        user=raw_url[raw_url.index('github.com')+1]
        clean_url='http://github.com/'+user
        clean_urls.append(clean_url)
    except ValueError:
        continue

clean_urls=list(set(clean_urls))

In [22]:
from parsel import Selector
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.common.exceptions import *



In [ ]:
df = pd.DataFrame(columns=['ISIM','OKUL','SIRKET','IS_DURUMU','YETENEK','PROFIL_URL','KAYNAK'])
driver = webdriver.Chrome('/opt/chromedriver')

for profile_url in clean_urls:
    driver.get(profile_url)
    sayfa_kaynagi = driver.page_source
    sel = Selector(text=sayfa_kaynagi)

    isim = sel.xpath('//*[@class="p-name vcard-fullname d-block overflow-hidden"]/text()').extract_first()
    sirket = sel.xpath('//*[@data-hovercard-type="organization"]/text()').extract_first()

    driver.get(profile_url+'?tab=repositories')
    sayfa_kaynagi = driver.page_source
    sel = Selector(text=sayfa_kaynagi)

    languages=sel.xpath('//*[@itemprop="programmingLanguage"]/text()').extract()
    languages = ','.join(list(set(languages)))
    
    sirket=np.nan if sirket==None else sirket
    languages=np.nan if languages==None else languages
    isim=np.nan if isim==None else isim
    
    yenisatir = {'ISIM':isim,'OKUL':np.nan,'YETENEK':languages,'SIRKET':sirket,'IS_DURUMU':np.nan,'PROFIL_URL':profile_url,'KAYNAK':'GitHub' }
    
    df = df.append(yenisatir,ignore_index=True)
